In [16]:
import pickle
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import resample
from datetime import datetime

In [17]:
unit_id = pd.read_csv("./data_dsmarket/unit_id.csv")
unit_id.head()

,year_week,id,units
0,2011-10,ACCESORIES_1_001_BOS_1,0
1,2011-10,ACCESORIES_1_001_BOS_2,0
2,2011-10,ACCESORIES_1_001_BOS_3,0
3,2011-10,ACCESORIES_1_001_NYC_1,0
4,2011-10,ACCESORIES_1_001_NYC_2,0


In [18]:
unit_id.shape

(8354260, 3)

In [19]:
item_sales = pd.read_csv("./data_dsmarket/item_sales.csv")
item_sales.head()

,id,item,category,department,store,store_code,region,d_1,d_2,d_3,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [20]:
cols = item_sales.columns.values.tolist()
cols_to_drop = cols[7:]
item_sales.drop(cols_to_drop, axis=1, inplace=True)

item_sales.head()

,id,item,category,department,store,store_code,region
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York


In [21]:
sales_calendar = pd.merge(
    left = unit_id,
    right = item_sales,
    how = "left",
    on = ["id"]
)

sales_calendar.head()

,year_week,id,units,item,category,department,store,store_code,region
0,2011-10,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston
1,2011-10,ACCESORIES_1_001_BOS_2,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Roxbury,BOS_2,Boston
2,2011-10,ACCESORIES_1_001_BOS_3,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Back_Bay,BOS_3,Boston
3,2011-10,ACCESORIES_1_001_NYC_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York
4,2011-10,ACCESORIES_1_001_NYC_2,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Harlem,NYC_2,New York


In [22]:
sales_calendar.shape

(8354260, 9)

In [23]:
date_id = pd.read_csv("./data_dsmarket/date_id.csv")
date_id.head()

,year_week,id,date
0,2011-10,ACCESORIES_1_001_BOS_1,2011-03-07
1,2011-10,ACCESORIES_1_001_BOS_2,2011-03-07
2,2011-10,ACCESORIES_1_001_BOS_3,2011-03-07
3,2011-10,ACCESORIES_1_001_NYC_1,2011-03-07
4,2011-10,ACCESORIES_1_001_NYC_2,2011-03-07


In [24]:
sales_calendar = pd.merge(
    left = sales_calendar,
    right = date_id,
    how = "left",
    on = ["id", "year_week" ]
)

sales_calendar.head()

,year_week,id,units,item,category,department,store,store_code,region,date
0,2011-10,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-03-07
1,2011-10,ACCESORIES_1_001_BOS_2,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Roxbury,BOS_2,Boston,2011-03-07
2,2011-10,ACCESORIES_1_001_BOS_3,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Back_Bay,BOS_3,Boston,2011-03-07
3,2011-10,ACCESORIES_1_001_NYC_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2011-03-07
4,2011-10,ACCESORIES_1_001_NYC_2,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Harlem,NYC_2,New York,2011-03-07


In [25]:
sales_calendar.shape

(8354260, 10)

In [26]:
item_prices = pd.read_csv("./data_dsmarket/item_prices.csv")
item_prices.head()

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328.0,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329.0,12.7414
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201330.0,10.9858
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201331.0,10.9858
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201332.0,10.9858


In [27]:
sales_calendar_prices = pd.merge(
    left = sales_calendar,
    right = item_prices,
    how = "left",
    on = ["id", "year_week" ]
)

sales_calendar_prices.head()

KeyError: 'id'

In [ ]:
sales_calendar_prices.shape

In [ ]:
sales_calendar_prices.to_csv("./data_dsmarket/sales_calendar_prices.csv")


In [ ]:
gb_year= sales_calendar_prices.groupby(["id", "item", "category", "department", "store", "region", "year"]).sum()
gb_year.reset_index(inplace=True)

gb_year.head(10)

In [ ]:
gb_year.drop("sell_price", axis=1, inplace=True)

In [ ]:
gb_year.shape

In [ ]:
gb_year.to_excel("./data_dsmarket/gb_year.xlsx")